<a href="https://colab.research.google.com/github/mnijhuis-dnb/Artificial_Intelligence_and_Machine_Learning_for_SupTech/blob/main/Tutorials/Tutorial%207%20NLTK%20and%20sentiment%20analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Artificial Intelligence and Machine Learning for SupTech  
Tutorial 7: NLTK and sentiment analysis

*	Constructing a bag of words
*	Classifying sentiments (positive/negative)
*	Example with financial news data

<br/>

15 March 2023  

**Instructors**  
Prof. Iman van Lelyveld (iman.van.lelyveld@vu.nl)<br/>
Dr. Michiel Nijhuis (m.nijhuis@dnb.nl)  

### Preparation

In [ ]:
import pandas as pd
import numpy as np
import re

for the natural language processing we need to download some word sets to be able to use some of the methods within the nltk package  

In [ ]:
import nltk

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('vader_lexicon')
!pip install twython 

Load articles from the FT. The datafile contains 436 articles from the Financial Times on the topic of Environmental, Social and Governance (ESG) between March 2021 and 2022.

In [ ]:
!gdown 1-4lW68b60PikNWQOfqc5qtriOEHC3_Kq

In [ ]:
df_articles = pd.read_csv('/content/df_articles.csv')
df_articles

### Extract tokens from the text
Current, the article is stored as `date`, `title` and `body`. Our focus will be on `body`. This is currently stored as a long string of text. See example below

In [ ]:
df_articles.loc[10, 'title']

In [ ]:
df_articles.loc[10, 'body']

This is not particularly useful as we want to extract meaningful words. For example, "green" or "inflation" or "rise". For this we first have to splice the `body` into individual tokens, that are separated by white spaces. The regular expression `'\w+'` means one or more alphanumeric characters. 

In [ ]:
from nltk.tokenize import RegexpTokenizer

# \w+ means at least one white space or line break character
regexp = RegexpTokenizer('\w+')

In [ ]:
df_articles['body_tokenize'] = df_articles['body'].apply(regexp.tokenize)

In [ ]:
df_articles.loc[0, 'body']

In [ ]:
df_articles.loc[0, 'body_tokenize']

Convert all words to lower case

In [ ]:
def convert_to_lowercase(list_of_tokens):
  return [tk.lower() for tk in list_of_tokens]

In [ ]:
convert_to_lowercase(['BaBaBa','bbbb','AAAA'])

In [ ]:
df_articles['body_tokenize'] = df_articles['body_tokenize'].apply(convert_to_lowercase)

In [ ]:
df_articles.loc[10, 'body_tokenize']

This is much better. But many words are not informative, which are what we call "stopwords". We can now remove stopwords. To be able to do this we should make use of a certain corpus. To view the nltk stopword corpus we can use the following code

In [ ]:
stopwords = nltk.corpus.stopwords.words("english")
stopwords

We want to filter out these words and only keep the meaningful ones. Moreover, short words that are one or two characters long can also be removed, just like words containing just numbers. Can you write a function that takes in a list of strings and removes the stopwords and returns the list without the stopwords?

In [ ]:
def remove_stopwords(tokens):
    non_stopwords_tokens = []
    for tk in tokens:
        if len(tk) < 3:
            # if the token is less than 3 characters long, jump to next token
            continue

        if any([a.isnumeric() for a in tk]):
            # if the token is numeric, e.g. '34' yields true
            continue

        if tk in stopwords:
            # if the token is a stopword, jump to next token
            continue

        # if no jumps happened, then add the token to the results
        non_stopwords_tokens.append(tk)

    return non_stopwords_tokens

To test the function you can use the following code to test it

In [ ]:
remove_stopwords([
    'ab',
    'abc',
    'only',
    '2020a',
    'England',
    'ESG',
    'of',
    'aaaaaaa'
])

Did you remove all of the stopwords correctly? The following function would also remove these words

In [ ]:
def remove_stopwords(tokens):
    non_stopwords_tokens = []
    for tk in tokens:
        if len(tk) < 3:
            # if the token is less than 3 characters long, jump to next token
            continue

        if any([a.isnumeric() for a in tk]):
            # if the token is numeric, e.g. '34' yields true
            continue

        if tk in stopwords:
            # if the token is a stopword, jump to next token
            continue

        # if no jumps happened, then add the token to the results
        non_stopwords_tokens.append(tk)

    return non_stopwords_tokens

In [ ]:
df_articles['body_tokenize_nonstop'] = df_articles['body_tokenize'].apply(remove_stopwords)

In [ ]:
df_articles.loc[10, 'body_tokenize_nonstop']

### Stemming and lemmatization

Great, now we need to use stemming or lemmatization to identify inflected forms of a word. I.e. `look` is the base form, but we usually see words like `looking`, `looks`, `looked` etc. Lemmatization goes a step further and also takes the context into account.

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer().lemmatize

In [ ]:
lemmatizer('bikes')

In [ ]:
lemmatizer('going')

Can you write a function to lemmatize the words within a list of words? 

In [ ]:
def lemmatize_each_word(words):
    lemmatized_words = []
    for word in words:
        lemmatized = lemmatizer(word)
        lemmatized_words.append(lemmatized)
    return lemmatized_words

To test the lemmatization you can use the following code

In [ ]:
lemmatize_each_word([
    'is',
    'the',
    'code',
    'working',
    'well',
    'or',
    'does',
    'it',
    'not',
    'work'
])

Did it go correctly, otherwise you can use the following function to lemmatize the words

In [ ]:
def lemmatize_each_word(words):
    lemmatized_words = []
    for word in words:
        lemmatized = lemmatizer(word)
        lemmatized_words.append(lemmatized)
    return lemmatized_words

In [ ]:
lemmatize_each_word(df_articles.loc[10, 'body_tokenize_nonstop'])

In [ ]:
df_articles['body_tokenize_nonstop_lemma'] = df_articles['body_tokenize_nonstop'].apply(lemmatize_each_word)

Now we can assign sentiments to the articles we have

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer().polarity_scores

In [ ]:
analyzer('good')

In [ ]:
analyzer('bad')

In [ ]:
analyzer('I like coffee')

For the sentiment we can also write a function to get the sentiment from a list of words

In [ ]:
def get_sentiment(list_of_words):
    words = ' '.join(list_of_words)
    analyzed = analyzer(words)
    return analyzed['compound']

In [ ]:
get_sentiment(['I', 'like', 'coffee'])

In [ ]:
df_articles['sentiment'] = df_articles['body_tokenize_nonstop_lemma'].apply(get_sentiment)

In [ ]:
df_articles['sentiment']

In [ ]:
df_articles.head()

We now have the desired results. We would like to plot these over time. So let's extract the `sentiment` column and plot it.

In [ ]:
df_articles['sentiment'].plot(figsize=[15,5])

This is very messy. There are several issues here that need to be addressed
1. The x-axis is wrong, it should be based on dates, not number of entries.
2. The series is very volatile, we are looking for a trend.

Let us first make our problem easier. `df_articles` contains all the data we constructed throughout the notebook. For plotting, however, we only need two columns: (a) the `date` column as the index and (b) the `sentiment` column for the actual time series values.

In [ ]:
df_articles.head(5)

Setting the dates correct

In [ ]:
sr_sentiment = df_articles.set_index('date')
sr_sentiment = sr_sentiment['sentiment']

In [ ]:
sr_sentiment.plot(figsize=[15,5], marker='o')

This looks identical as the previous figure, only that the x-labels are different. If we look closely, however, the dates are not actual dates. Instead, these are texts. Let's have a look at the index of `sr_sentiment`

In [ ]:
sr_sentiment.index

The `dtype='object'` means that the index is just a list of string values. We want to have dates.

In [ ]:
sr_sentiment.index = pd.to_datetime(sr_sentiment.index)
sr_sentiment.index

In [ ]:
sr_sentiment.plot(figsize=[15,5], marker='o', lw=0)

What's different? A lot of diagonal lines appear that seem to jump between data points. This is usually a sign of interpolation and missing values. 

Can we see the missing values? Let's have look at `sr_sentiment`. Notice anything strange?

In [ ]:
sr_sentiment

Can you make a nicer graph of how the sentiment varies over time?

In [ ]:
sr_sentiment.groupby([sr_sentiment.year, sr_sentiment.week]).mean().plot(figsize=[15,5])